In [2]:
import numpy as np

def calculate_metrics(true_masks, predicted_masks, threshold=0.5):
    true_masks = np.asarray(true_masks, dtype=bool)
    predicted_masks = np.asarray(predicted_masks > threshold, dtype=bool)
    
    # Приведение формы предсказанных масок к форме истинных масок, если необходимо
    if predicted_masks.shape[-1] == 1:
        predicted_masks = np.squeeze(predicted_masks, axis=-1)
    
    intersection = np.sum(np.logical_and(true_masks, predicted_masks))
    union = np.sum(np.logical_or(true_masks, predicted_masks))
    
    # Dice Coefficient
    dice = (2.0 * intersection) / (np.sum(true_masks) + np.sum(predicted_masks))
    
    # Precision
    if np.sum(predicted_masks) != 0:
        precision = intersection / np.sum(predicted_masks)
    else:
        precision = -1
    
    # IoU (Intersection over Union)
    if union != 0:
        iou = intersection / union
    else:
        iou = -1
    
    # Recall
    recall = intersection / np.sum(true_masks)
    
    # F1-score
    if precision + recall != 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = -1
    
    return dice, precision, iou, f1_score

In [3]:
import os
import cv2
import numpy as np

test_images_src = 'DIPLOMA/Dataset_for_test/images/'
test_masks_src = 'DIPLOMA/Dataset_for_test/masks/'

def load_masks_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return images

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images



# Загрузка изображений и масок
test_images = load_images_from_folder(test_images_src)
true_masks = load_masks_from_folder(test_masks_src)

true_masks = np.array(true_masks)
test_images = np.array(test_images)

In [4]:
len(true_masks), len(test_images)

(309, 309)

In [5]:
true_masks.shape, test_images.shape

((309, 256, 256), (309, 256, 256, 3))

In [6]:
from keras.models import load_model
import numpy as np

model_path = "segnet_256_auged.h5"
model = load_model(model_path)

predicted_masks_list = []

for image in test_images:
    predicted_mask = model.predict(np.expand_dims(image, axis=0))
    predicted_mask = np.squeeze(predicted_mask, axis=0)
    predicted_masks_list.append(predicted_mask)
    

predicted_masks = np.array(predicted_masks_list)

print("Размерность массива предсказанных масок:", predicted_masks.shape)

Размерность массива предсказанных масок: (309, 256, 256, 2)


In [7]:
from keras.models import load_model

predicted_masks = predicted_masks[:, :, :, 1]  # Используем только второй канал (например, для объекта)
predicted_masks = np.round(predicted_masks)   # Преобразуем вероятности в бинарные значения

# Проведём изучение метрик
dice, precision, iou, f1_score = calculate_metrics(true_masks, predicted_masks)
print("Dice Coefficient:", dice)
print("Precision:", precision)
print("IoU:", iou)
print("F1-score:", f1_score)

Dice Coefficient: 0.5253069616237545
Precision: 0.6532931022147103
IoU: 0.3562144446020843
F1-score: 0.5253069616237545
